<a href="https://colab.research.google.com/github/daisuke08253649/DeepLearning/blob/main/emotion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('./drive/MyDrive/DeepLearning/SIGNATE/emotion_classification/train_master.tsv', sep='\t')

train_datas = df[['id', 'expression']]

emotions = {'neutral' : 0, 'happy' : 1, 'sad' : 2, 'angry' : 3}

train_datas['expression'] = train_datas['expression'].replace(emotions)

print(train_datas)

num_classes = len(emotions)

image_name_list = train_datas['id'].values
label_list = train_datas['expression'].values


image_label = np.eye(num_classes)[label_list]

print(image_label)


x_train, x_val, y_train, y_val = train_test_split(image_name_list, image_label, test_size=0.25, stratify=image_label, random_state=42)

print('学習データ数',len(x_train))
print('検証データ数',len(x_val))
print('学習データラベル数')
print(np.unique(y_train.argmax(axis=1), return_counts=True))
print('検証データラベル数')
print(np.unique(y_val.argmax(axis=1), return_counts=True))

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

train_image_dir = './drive/MyDrive/DeepLearning/SIGNATE/emotion_classification/train/train/'
train_image_list = os.listdir(train_image_dir)

image_list = []

for image_path in train_image_list:
    image = Image.open(train_image_dir + image_path)
    image = np.array(image)
    image_list.append(image)

print(len(image_list))
plt.imshow(image)
plt.show()

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


transform = {
    'train': transforms.Compose([
        transforms.Resize((120, 120)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),

    'val': transforms.Compose([
        transforms.Resize((120, 120)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

#image_transformed = transform['train'](image)

class EmotionDataset(Dataset):
    def __init__(self, image_name_list, label_list, phase):
        self.image_name_list = image_name_list
        self.label_list = label_list
        self.phase = phase
        self.transform = transform

    def __len__(self):
        return len(self.image_name_list)

    def __getitem__(self, index):
        image_name = self.image_name_list[index]
        image = Image.open(train_image_dir + image_name)

        image = self.transform[self.phase](image)
        label = self.label_list[index]

        return image, label

train_dataset = EmotionDataset(x_train, y_train, phase='train')
val_dataset = EmotionDataset(x_val, y_train, phase='val')

print(val_dataset.__getitem__(0)[0].shape)

batch_size = 40
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size, shuffle=False)

dataloaders_dict = {
    'train': train_dataloader,
    'val': val_dataloader
}

In [ ]:
from torch import nn, optim
import torch

net = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

net.conv1 = nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1, bias=False)
net.fc = nn.Linear(in_features=512, out_features=4, bias=True)

print(net)

In [ ]:
def train_model(net, epochs, criterion, optimizer):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    net.to(device)

    best_iou = 0.0

    for epoch in range(epochs):
        print(f'Epoch: {epoch+1} / {epochs}')
        print('--------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0
            pred_list = []
            true_list = []

            for images, labels in dataloaders_dict[phase]:
                images = images.float().to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=='train'):
                    outputs = net(images)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * images.size(0)

                    pred = preds.to('cpu').numpy()
                    pred_list.extend(pred)

                    labels = labels.to('cpu').numpy()
                    true_list.extend(labels)

            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f}')

        print(pred_list)
        print(true_list)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

epochs = 100

train_model(net, epochs, criterion, optimizer)
#torch.save(net, 'model.pth')

In [ ]:
test_image_dir = './drive/MyDrive/DeepLearning/SIGNATE/emotion_classification/test/test/'
test_image_list = os.listdir(test_image_dir)

for test_image_name in test_image_list:
    test_image = Image.open(test_image_dir + test_image_name)

plt.imshow(test_image)
plt.show()

In [ ]:
test_transform = {
    'test': transforms.Compose([
        transforms.Resize((120, 120)),
        transforms.ToTensor()
    ])
}

class Test_EmotionDataset(Dataset):
    def __init__(self, test_image_dir, test_image_list, test_label_df, phase):
        self.test_image_dir = test_image_dir
        self.test_image_list = test_image_list
        self.test_label_df = test_label_df
        self.phase = phase
        self.test_transform = test_transform

    def __len__(self):
        return len(self.test_image_list)

    def __getitem__(self, index):
        test_image = Image.open(self.test_image_dir + self.test_image_list[index])
        image = self.test_transform[self.phase](test_image)

        label = self.test_label_df[index]

        return image, label

test_dataset = Test_EmotionDataset(test_image_dir, test_image_list, test_label_df, phase='test')

batch_size = 40
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

In [ ]:
def test_model(net, test_dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.load(net)
    model.to(device)

    pred_list = []

    model.eval()
    with torch.no_grad():
        for image, label in test_dataloader:
            image = image.to(device)
            outputs = model(image)
            _, predict = torch.max(outputs, 1)
            predict = predict.to('cpu').numpy()
            pred_list.extend(predict)

    return pred_list

In [ ]:
net = './drive/MyDrive/DeepLearning/SIGNATE/emotion_classification/model.pth'

predict_list = test_model(net, test_dataloader)
print(predict_list)

In [ ]:
test_df = pd.read_csv('./drive/MyDrive/DeepLearning/SIGNATE/emotion_classification/sample_submit (4).csv', header=None)

emotions = {0 : 'neutral', 1 : 'happy', 2 : 'sad', 3 : 'angry'}

test_image_df = test_df.iloc[:, 0]
test_label_df = test_df.iloc[:, 1]

for i in range(len(test_label_df)):
    test_label_df[i] = predict_list[i]
    pred_label_df = test_label_df.replace(emotions)


print(test_image_df)
print(pred_label_df)

test_df.iloc[:, 1] = pred_label_df
test_df.to_csv('./drive/MyDrive/DeepLearning/SIGNATE/emotion_classification/submit.csv', header=None, index=None)